In [34]:
import pandas as pd
df = pd.read_csv('/Users/irisz/Downloads/Jupyter/gap-coreference-master/gap-combine.tsv', sep = '\t')
df


,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,True,Pauline,207,False,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,True,Bernard Leach,251,False,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,False,De la Sota,246,True,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,False,Henry Rosenthal,336,True,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,False,Rivera,294,True,http://en.wikipedia.org/wiki/Jessica_Rivera
...,...,...,...,...,...,...,...,...,...,...,...
4448,validation-450,"He then agrees to name the gargoyle Goldie, af...",He,305,Lucien,252,False,Abel,264,False,http://en.wikipedia.org/wiki/Goldie_(DC_Comics)
4449,validation-451,"Disgusted with the family's ``mendacity'', Bri...",she,365,Maggie,242,False,Mae,257,False,http://en.wikipedia.org/wiki/Cat_on_a_Hot_Tin_...
4450,validation-452,She manipulates Michael into giving her custod...,she,306,Scarlett,255,False,Alice,291,True,http://en.wikipedia.org/wiki/Michael_Moon_(Eas...
4451,validation-453,"On April 4, 1986, Donal Henahan wrote in the N...",her,330,Aida,250,False,Miss Millo,294,True,http://en.wikipedia.org/wiki/Aprile_Millo


In [35]:
import numpy as np
sLength = len(df)
df['A-to-pron-chr'] = pd.Series(np.random.randn(sLength), index=df.index)
df['B-to-pron-chr'] = pd.Series(np.random.randn(sLength), index=df.index)
df['A-to-pron-tk'] = pd.Series(np.random.randn(sLength), index=df.index)
df['B-to-pron-tk'] = pd.Series(np.random.randn(sLength), index=df.index)

df['A-to-pron-chr'] = df['Pronoun-offset'] - df['A-offset']
df['B-to-pron-chr'] = df['Pronoun-offset'] - df['B-offset']

df['A-tk-index'] = pd.Series(np.random.randn(sLength), index=df.index)
df['B-tk-index'] = pd.Series(np.random.randn(sLength), index=df.index)
df['Pronoun-tk-index'] = pd.Series(np.random.randn(sLength), index=df.index)


In [36]:
import spacy
nlp = spacy.load('en_core_web_sm')
df['text_tk'] = df['Text'].apply(lambda x: nlp(x))

In [37]:
df['A-tk-index'] = df.apply(lambda x: [token.idx for token in x.text_tk].index(x['A-offset']), axis=1)


In [38]:
df['A-tk-index'] = df.apply(lambda x: [token.idx for token in x.text_tk].index(x['A-offset']), axis=1)
df['B-tk-index'] = df.apply(lambda x: [token.idx for token in x.text_tk].index(x['B-offset']), axis=1)
df['Pronoun-tk-index'] = df.apply(lambda x: [token.idx for token in x.text_tk].index(x['Pronoun-offset']), axis=1)



In [39]:
df['A-to-pron-tk'] = df['Pronoun-tk-index'] - df['A-tk-index']
df['B-to-pron-tk'] = df['Pronoun-tk-index'] - df['B-tk-index']



In [40]:
def get_dep(index_start, doc, entity):
    length = len(entity.split(' '))
    index_end = index_start + length -1
    dep = doc[index_end].dep_
    return dep

df['A-dep'] = pd.Series(np.random.randn(sLength), index=df.index)
df['B-dep'] = pd.Series(np.random.randn(sLength), index=df.index)
df['Pronoun-dep'] = pd.Series(np.random.randn(sLength), index=df.index)



In [41]:
def commasbetween(section,nameindex,pronounindex):
    slice = section[nameindex+1:pronounindex]
    commacount=0
    for token in slice:
        if token.pos_=='PUNCT':
            if token.text == ',':
                commacount +=1
    return commacount

def periodsbetween(section,nameindex,pronounindex):
    slice = section[nameindex+1:pronounindex]
    periodcount=0
    for token in slice:
        if token.pos_=='PUNCT':
            if token.text == '.':
                periodcount +=1
    return periodcount


def nounsbetween(section,nameindex,pronounindex):
    slice = section[nameindex+1:pronounindex]
    namecount = 0
    for token in slice:
        if (token.pos_ == 'PROPN' or token.pos_ == 'NOUN') and token.dep_ != 'compound':
            namecount+=1
    return namecount

def names(section,namestring):
    namecount=0
    nameparts = namestring.split(' ')
    for token in section:
        for name in nameparts:
            if token.text == name and token.dep_ != 'compound':
                namecount+=1
    return namecount

df['Commas-A-Pron'] = pd.Series(np.random.randn(sLength), index=df.index)
df['Commas-B-Pron'] = pd.Series(np.random.randn(sLength), index=df.index)
df['Commas-A-Pron'] = df.apply(lambda x: commasbetween(x.text_tk,x['A-tk-index'],x['Pronoun-tk-index']), axis=1)
df['Commas-B-Pron'] = df.apply(lambda x: commasbetween(x.text_tk,x['B-tk-index'],x['Pronoun-tk-index']), axis=1)
df['Period-A-Pron'] = pd.Series(np.random.randn(sLength), index=df.index)
df['Period-B-Pron'] = pd.Series(np.random.randn(sLength), index=df.index)
df['Period-A-Pron'] = df.apply(lambda x: periodsbetween(x.text_tk,x['A-tk-index'],x['Pronoun-tk-index']), axis=1)
df['Period-B-Pron'] = df.apply(lambda x: periodsbetween(x.text_tk,x['B-tk-index'],x['Pronoun-tk-index']), axis=1)

df['Nouns-A-Pronoun'] = pd.Series(np.random.randn(sLength), index=df.index)
df['Nouns-A-Pronoun'] = df.apply(lambda x: nounsbetween(x.text_tk,x['A-tk-index']+len(x['A'].split(' ')),x['Pronoun-tk-index']), axis=1)
df['Nouns-B-Pronoun'] = pd.Series(np.random.randn(sLength), index=df.index)
df['Nouns-B-Pronoun'] = df.apply(lambda x: nounsbetween(x.text_tk,x['B-tk-index']+len(x['B'].split(' ')),x['Pronoun-tk-index']), axis=1)

df['A-Mentions'] = pd.Series(np.random.randn(sLength), index=df.index)
df['A-Mentions'] = df.apply(lambda x: names(x.text_tk,x['A']), axis=1)
df['B-Mentions'] = pd.Series(np.random.randn(sLength), index=df.index)
df['B-Mentions'] = df.apply(lambda x: names(x.text_tk,x['B']), axis=1)



In [42]:
df1 = df

In [43]:

#generate all the dep features for A B Pronoun
for i in range(len(df1)):
    df1['A-dep'][i] = get_dep(df1['A-tk-index'][i], df1['text_tk'][i],df1['A'][i])
    df1['B-dep'][i] = get_dep(df1['B-tk-index'][i], df1['text_tk'][i],df1['B'][i])
    df1['Pronoun-dep'][i] = get_dep(df1['Pronoun-tk-index'][i], df1['text_tk'][i],df1['Pronoun'][i])
    
df1['nsubj-A'] = pd.Series(0, index=df.index)
df1['pobj-A'] = pd.Series(0, index=df.index)
df1['poss-A'] = pd.Series(0, index=df.index)
df1['appos-A'] = pd.Series(0, index=df.index)
df1['dobj-A'] = pd.Series(0, index=df.index)
df1['nsubjpass-A'] = pd.Series(0, index=df.index)

df1['nsubj-B'] = pd.Series(0, index=df.index)
df1['pobj-B'] = pd.Series(0, index=df.index)
df1['poss-B'] = pd.Series(0, index=df.index)
df1['appos-B'] = pd.Series(0, index=df.index)
df1['dobj-B'] = pd.Series(0, index=df.index)
df1['nsubjpass-B'] = pd.Series(0, index=df.index)

df1['nsubj-Pronoun'] = pd.Series(0, index=df.index)
df1['pobj-Pronoun'] = pd.Series(0, index=df.index)
df1['poss-Pronoun'] = pd.Series(0, index=df.index)
df1['appos-Pronoun'] = pd.Series(0, index=df.index)
df1['dobj-Pronoun'] = pd.Series(0, index=df.index)
df1['nsubjpass-Pronoun'] = pd.Series(0, index=df.index)

def feature_match(feature, aim):
    if feature == aim:
        return 1
    else:
        return 0
    
feature_list = ['nsubj','pobj','poss','appos','dobj','nsubjpass']
aims = ['A','B','Pronoun']

for aim in aims:
    aim_dep_col = aim + '-' + 'dep'
    for feature in feature_list:
        feature_col = feature + '-' + aim
        for i in range(len(df1)):
            df1[feature_col][i] = feature_match(feature,df1[aim_dep_col][i])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the 

In [44]:
df1

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,...,poss-B,appos-B,dobj-B,nsubjpass-B,nsubj-Pronoun,pobj-Pronoun,poss-Pronoun,appos-Pronoun,dobj-Pronoun,nsubjpass-Pronoun
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,True,Pauline,207,False,...,1,0,0,0,0,0,1,0,0,0
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,True,Bernard Leach,251,False,...,0,0,0,0,0,0,1,0,0,0
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,False,De la Sota,246,True,...,0,0,1,0,0,0,1,0,0,0
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,False,Henry Rosenthal,336,True,...,0,0,0,0,0,0,1,0,0,0
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,False,Rivera,294,True,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4448,validation-450,"He then agrees to name the gargoyle Goldie, af...",He,305,Lucien,252,False,Abel,264,False,...,0,0,1,0,1,0,0,0,0,0
4449,validation-451,"Disgusted with the family's ``mendacity'', Bri...",she,365,Maggie,242,False,Mae,257,False,...,0,0,0,0,0,0,0,0,0,1
4450,validation-452,She manipulates Michael into giving her custod...,she,306,Scarlett,255,False,Alice,291,True,...,0,0,1,0,1,0,0,0,0,0
4451,validation-453,"On April 4, 1986, Donal Henahan wrote in the N...",her,330,Aida,250,False,Miss Millo,294,True,...,0,0,0,0,1,0,0,0,0,0


In [45]:
cols = ['Pronoun-offset','A-offset','A-coref','B-offset','B-coref','A-to-pron-chr','B-to-pron-chr','A-to-pron-tk','B-to-pron-tk','A-tk-index','B-tk-index','Pronoun-tk-index','nsubj-A','pobj-A','poss-A','appos-A','dobj-A','nsubjpass-A','nsubj-B','pobj-B','poss-B','appos-B','dobj-B','nsubjpass-B','nsubj-Pronoun','pobj-Pronoun','poss-Pronoun','appos-Pronoun','dobj-Pronoun','nsubjpass-Pronoun','Commas-A-Pron','Commas-B-Pron','Period-A-Pron','Period-B-Pron','Nouns-A-Pronoun','Nouns-B-Pronoun','A-Mentions','B-Mentions']
df2 = df1[cols]


In [46]:
def TF_number(TF):
    if TF is True:
        return 1
    else:
        return 0
    
df2['A-coref'] = df2['A-coref'].apply(lambda x: TF_number(x))
df2['B-coref'] = df2['B-coref'].apply(lambda x: TF_number(x))


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [47]:
df2.to_csv('/Users/irisz/Downloads/Jupyter/gap-coreference-master/gap-nlp-classifier-combine.tsv', sep='\t')


In [48]:
df2

,Pronoun-offset,A-offset,A-coref,B-offset,B-coref,A-to-pron-chr,B-to-pron-chr,A-to-pron-tk,B-to-pron-tk,A-tk-index,...,dobj-Pronoun,nsubjpass-Pronoun,Commas-A-Pron,Commas-B-Pron,Period-A-Pron,Period-B-Pron,Nouns-A-Pronoun,Nouns-B-Pronoun,A-Mentions,B-Mentions
0,274,191,1,207,0,83,67,18,15,39,...,0,0,1,0,1,1,6,5,1,2
1,284,228,1,251,0,56,33,10,7,46,...,0,0,0,0,1,1,1,0,1,1
2,265,173,0,246,1,92,19,19,5,35,...,0,0,1,0,0,0,3,0,1,3
3,321,174,0,336,1,147,-15,31,-4,35,...,0,0,2,0,1,0,9,0,1,1
4,437,219,0,294,1,218,143,43,28,42,...,0,0,6,3,1,1,10,7,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4448,305,252,0,264,0,53,41,10,8,56,...,0,0,0,0,1,1,3,2,1,2
4449,365,242,0,257,0,123,108,28,24,50,...,0,1,4,2,0,0,9,7,1,1
4450,306,255,0,291,1,51,15,9,3,46,...,0,0,1,0,0,0,2,0,4,4
4451,330,250,0,294,1,80,36,19,8,56,...,0,0,2,0,0,0,4,1,1,3
